In [ ]:
import matplotlib.pyplot as plt
from binary import img_to_binary
from add_contours import get_highlight_edge_img

In [ ]:
plt.rcParams["axes.labelsize"] = 25
plt.rcParams["axes.titlesize"] = 20
plt.rcParams["axes.linewidth"] = 1
plt.rcParams["axes.titlepad"] = 25
plt.rcParams["axes.labelpad"] = 10
plt.rc("xtick.major", width=2, size=5, pad=10)
plt.rc("ytick.major", width=2, size=5, pad=10)
plt.rc("xtick", labelsize=15, direction="out")
plt.rc("ytick", labelsize=15, direction="out")

In [ ]:
def show_img(img, title):
    fig, ax = plt.subplots()

    ax.imshow(img, cmap="gray")
    ax.set_title(title)
    ax.set_xlabel("$X$")
    ax.set_ylabel("$Y$")
    ax.set_xlim(0, img.shape[1])
    ax.set_ylim(img.shape[0], 0)
    fig.show()

    return

In [ ]:
input_filepath = "../../../data/POM/imgs/crack.jpeg"

In [ ]:
binary_img = img_to_binary(input_filepath, threshold=70)

In [ ]:
show_img(binary_img, title="crack binary image")

In [ ]:
highlight_edge_img = get_highlight_edge_img(
    binary_img, min_contour_size=0, line_width=10
)
show_img(highlight_edge_img, title="crack highlight edge image")